In [1]:
# Load env variables and create client
from anthropic import Anthropic
from anthropic.types import ToolParam, Message
from datetime import datetime, timedelta
from dotenv import load_dotenv
import json
load_dotenv()

client = Anthropic()
model = "claude-3-7-sonnet-latest"

In [29]:
# Helper functions
def add_user_message(messages, message):
    user_message = {
        "role": "user", 
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(user_message)


def add_assistant_message(messages, message):
    assistant_message = {
        "role": "assistant", 
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[], tools=None, tool_choice=None):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system
    if tools:
        params["tools"] = tools
    if tool_choice:
        params["tool_choice"] = tool_choice

    message = client.messages.create(**params)
    return message

def text_from_message(message):
    return "\n".join(
        [block.text for block in message.content if block.type == "text"]
    )

In [39]:
# Tools and Schemas

from datetime import datetime, timedelta


def add_duration_to_datetime(
    datetime_str, duration=0, unit="days", input_format="%Y-%m-%d"
):
    date = datetime.strptime(datetime_str, input_format)

    if unit == "seconds":
        new_date = date + timedelta(seconds=duration)
    elif unit == "minutes":
        new_date = date + timedelta(minutes=duration)
    elif unit == "hours":
        new_date = date + timedelta(hours=duration)
    elif unit == "days":
        new_date = date + timedelta(days=duration)
    elif unit == "weeks":
        new_date = date + timedelta(weeks=duration)
    elif unit == "months":
        month = date.month + duration
        year = date.year + month // 12
        month = month % 12
        if month == 0:
            month = 12
            year -= 1
        day = min(
            date.day,
            [
                31,
                29
                if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)
                else 28,
                31,
                30,
                31,
                30,
                31,
                31,
                30,
                31,
                30,
                31,
            ][month - 1],
        )
        new_date = date.replace(year=year, month=month, day=day)
    elif unit == "years":
        new_date = date.replace(year=date.year + duration)
    else:
        raise ValueError(f"Unsupported time unit: {unit}")

    return new_date.strftime("%A, %B %d, %Y %I:%M:%S %p")


def set_reminder(content, timestamp):
    print(
        f"----\nSetting the following reminder for {timestamp}:\n{content}\n----"
    )


article_summary_schema = {
    "name": "article_summary",
    "description": "Creates a summary of an article with its key insights. Use this tool when you need to generate a structured summary of an article, research paper, or any textual content. The tool requires the article's title, author name, and a list of the most important insights or takeaways from the content. Each insight should be a concise statement capturing a significant point from the article.",
    "input_schema": {
        "type": "object",
        "properties": {
            "title": {
                "type": "string",
                "description": "The title of the article being summarized.",
            },
            "author": {
                "type": "string",
                "description": "The name of the author who wrote the article.",
            },
            "key_insights": {
                "type": "array",
                "items": {"type": "string"},
                "description": "A list of the most important takeaways or insights from the article. Each insight should be a complete, concise statement.",
            },
        },
        "required": ["title", "author", "key_insights"],
    },
}
    
add_duration_to_datetime_schema = {
    "name": "add_duration_to_datetime",
    "description": "Adds a specified duration to a datetime string and returns the resulting datetime in a detailed format. This tool converts an input datetime string to a Python datetime object, adds the specified duration in the requested unit, and returns a formatted string of the resulting datetime. It handles various time units including seconds, minutes, hours, days, weeks, months, and years, with special handling for month and year calculations to account for varying month lengths and leap years. The output is always returned in a detailed format that includes the day of the week, month name, day, year, and time with AM/PM indicator (e.g., 'Thursday, April 03, 2025 10:30:00 AM').",
    "input_schema": {
        "type": "object",
        "properties": {
            "datetime_str": {
                "type": "string",
                "description": "The input datetime string to which the duration will be added. This should be formatted according to the input_format parameter.",
            },
            "duration": {
                "type": "number",
                "description": "The amount of time to add to the datetime. Can be positive (for future dates) or negative (for past dates). Defaults to 0.",
            },
            "unit": {
                "type": "string",
                "description": "The unit of time for the duration. Must be one of: 'seconds', 'minutes', 'hours', 'days', 'weeks', 'months', or 'years'. Defaults to 'days'.",
            },
            "input_format": {
                "type": "string",
                "description": "The format string for parsing the input datetime_str, using Python's strptime format codes. For example, '%Y-%m-%d' for ISO format dates like '2025-04-03'. Defaults to '%Y-%m-%d'.",
            },
        },
        "required": ["datetime_str"],
    },
}

set_reminder_schema = {
    "name": "set_reminder",
    "description": "Creates a timed reminder that will notify the user at the specified time with the provided content. This tool schedules a notification to be delivered to the user at the exact timestamp provided. It should be used when a user wants to be reminded about something specific at a future point in time. The reminder system will store the content and timestamp, then trigger a notification through the user's preferred notification channels (mobile alerts, email, etc.) when the specified time arrives. Reminders are persisted even if the application is closed or the device is restarted. Users can rely on this function for important time-sensitive notifications such as meetings, tasks, medication schedules, or any other time-bound activities.",
    "input_schema": {
        "type": "object",
        "properties": {
            "content": {
                "type": "string",
                "description": "The message text that will be displayed in the reminder notification. This should contain the specific information the user wants to be reminded about, such as 'Take medication', 'Join video call with team', or 'Pay utility bills'.",
            },
            "timestamp": {
                "type": "string",
                "description": "The exact date and time when the reminder should be triggered, formatted as an ISO 8601 timestamp (YYYY-MM-DDTHH:MM:SS) or a Unix timestamp. The system handles all timezone processing internally, ensuring reminders are triggered at the correct time regardless of where the user is located. Users can simply specify the desired time without worrying about timezone configurations.",
            },
        },
        "required": ["content", "timestamp"],
    },
}

batch_tool_schema = {
    "name": "batch_tool",
    "description": "Invoke multiple other tool calls simultaneously",
    "input_schema": {
        "type": "object",
        "properties": {
            "invocations": {
                "type": "array",
                "description": "The tool calls to invoke",
                "items": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "The name of the tool to invoke",
                        },
                        "arguments": {
                            "type": "string",
                            "description": "The arguments to the tool, encoded as a JSON string",
                        },
                    },
                    "required": ["name", "arguments"],
                },
            }
        },
        "required": ["invocations"],
    },
}

pass


In [40]:
def get_current_datetime(date_format="%Y:%m:%d %H:%M:%S"):
    if not date_format:
        raise ValueError("date_format cannot be empty")
    return datetime.now().strftime(date_format)

get_current_datetime_schema = ToolParam({
  "name": "get_current_datetime",
  "description": "Get the current date and time formatted according to a specified format string. Returns the current system datetime in the requested format. This tool is useful for timestamping, scheduling, logging, or any task requiring the current date/time.",
  "input_schema": {
    "type": "object",
    "properties": {
      "date_format": {
        "type": "string",
        "description": "The format string for the datetime output using Python strftime format codes. Common formats include: '%Y-%m-%d %H:%M:%S' (2024-01-15 14:30:45), '%Y-%m-%d' (2024-01-15), '%H:%M:%S' (14:30:45), '%B %d, %Y' (January 15, 2024), '%A, %B %d, %Y at %I:%M %p' (Monday, January 15, 2024 at 02:30 PM). See Python strftime documentation for all available format codes.",
        "default": "%Y-%m-%d %H:%M:%S",
        "examples": [
          "%Y-%m-%d %H:%M:%S",
          "%Y-%m-%d",
          "%H:%M:%S",
          "%B %d, %Y",
          "%A, %B %d, %Y at %I:%M %p",
          "%m/%d/%Y",
          "%d/%m/%Y %H:%M",
          "%Y%m%d_%H%M%S"
        ]
      }
    },
    "required": []
  }
})

get_current_datetime()

'2025:07:14 12:42:34'

In [41]:
messages=[]
messages.append({
    "role":"user",
    "content":"What is the exact time formatted %H:%M:%S"
})

response = client.messages.create(
    messages=messages,
    model = model,
    max_tokens = 1000,
)
response

Message(id='msg_019Dy36rnntfqodri6pmYgRp', content=[TextBlock(citations=None, text="The current exact time formatted as %H:%M:%S would be the hours, minutes, and seconds in 24-hour format. However, I don't have access to the current time as I don't have real-time clock capabilities. \n\nThe %H:%M:%S format would display time as:\n- %H: Hours in 24-hour format (00-23)\n- %M: Minutes (00-59)\n- %S: Seconds (00-59)\n\nFor example, 3:45:22 PM would be displayed as 15:45:22 in this format.", type='text')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=19, output_tokens=135, server_tool_use=None, service_tier='standard'))

In [42]:
messages=[]
messages.append({
    "role":"user",
    "content":"What is the exact time formatted %H:%M:%S"
})

response = client.messages.create(
    messages=messages,
    model = model,
    max_tokens = 1000,
    tools=[get_current_datetime_schema,]
)
response

Message(id='msg_01QuB5HyqvpHZbyNewpeCfT8', content=[TextBlock(citations=None, text="I'll get the current time for you in the requested format.", type='text'), ToolUseBlock(id='toolu_01ErFhZMg6CqkxCQsfc6kjru', input={'date_format': '%H:%M:%S'}, name='get_current_datetime', type='tool_use')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=675, output_tokens=76, server_tool_use=None, service_tier='standard'))

In [43]:
get_current_datetime(**response.content[1].input)

'12:42:38'

In [44]:
messages.append({
    "role": "assistant",
    "content": response.content
})

messages.append({
    "role":"user",
    "content":[{
        "type":"tool_result",
        "tool_use_id":response.content[1].id,
        "content":"10:14:28",
        "is_error":False,
    }]
})


In [45]:
response.content[1].id

'toolu_01ErFhZMg6CqkxCQsfc6kjru'

In [46]:
messages

[{'role': 'user', 'content': 'What is the exact time formatted %H:%M:%S'},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text="I'll get the current time for you in the requested format.", type='text'),
   ToolUseBlock(id='toolu_01ErFhZMg6CqkxCQsfc6kjru', input={'date_format': '%H:%M:%S'}, name='get_current_datetime', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01ErFhZMg6CqkxCQsfc6kjru',
    'content': '10:14:28',
    'is_error': False}]}]

In [47]:
response2 = client.messages.create(
    model=model,
    max_tokens=1000,
    messages=messages,
    tools=[get_current_datetime_schema],
)
response2

Message(id='msg_0186CK6egQEkQdm5QuP2dWws', content=[TextBlock(citations=None, text='The current time is 10:14:28 in the %H:%M:%S format (hours:minutes:seconds in 24-hour notation).', type='text')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=767, output_tokens=37, server_tool_use=None, service_tier='standard'))

In [48]:
messages=[]
add_user_message(messages, "What is the exact time formatted %H:%M:%S")

response = client.messages.create(
    messages=messages,
    model = model,
    max_tokens = 1000,
    tools=[get_current_datetime_schema,]
)
add_assistant_message(messages, response.content)

messages

[{'role': 'user', 'content': 'What is the exact time formatted %H:%M:%S'},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text="I'll get the current time for you in the requested format.", type='text'),
   ToolUseBlock(id='toolu_01RjBZhFHMqicGYgz9pxvhi9', input={'date_format': '%H:%M:%S'}, name='get_current_datetime', type='tool_use')]}]

In [49]:
response

Message(id='msg_01WPWNrECVfuw6oP7SZCPxcK', content=[TextBlock(citations=None, text="I'll get the current time for you in the requested format.", type='text'), ToolUseBlock(id='toolu_01RjBZhFHMqicGYgz9pxvhi9', input={'date_format': '%H:%M:%S'}, name='get_current_datetime', type='tool_use')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=675, output_tokens=76, server_tool_use=None, service_tier='standard'))

In [50]:
def run_batch(invocations=[]):
    batch_output = []
    
    for invocation in invocations:
        name = invocation["name"]
        args = json.loads(invocation["arguments"])
        
        tool_output = run_tool(name, args)
        
        batch_output.append({
            "tool_name": name,
            "output": tool_output
        })
    
    return batch_output

def run_tool(tool_name, tool_input):
    if tool_name == "get_current_datetime":
        return get_current_datetime(**tool_input)
    elif tool_name == "add_duration_to_datetime":
        return add_duration_to_datetime(**tool_input)
    elif tool_name == "set_reminder":
        return set_reminder(**tool_input)
    elif tool_name == "run_batch":
        return set_reminder(**tool_input)
        
def run_tools(message):
    tool_requests = [
        block for block in message.content if block.type == "tool_use"
    ]

    tool_result_blocks = []
    
    for request in tool_requests:
        try:
            output = run_tool(request.name, request.input)
    
            response = {
                        "type":"tool_result",
                        "tool_use_id":request.id,
                        "content":json.dumps(output),
                        "is_error":False,
            }
        except Exception as e:
            response = {
                        "type":"tool_result",
                        "tool_use_id":request.id,
                        "content":f"Error: {e}",
                        "is_error":True,
            }
        tool_result_blocks.append(response)
    return tool_result_blocks

In [51]:
def run_conversation(messages):
    while True:
        response = chat(messages, tools=[get_current_datetime_schema])
        add_assistant_message(messages, response)
        print(text_from_message(response))
        
        if response.stop_reason != "tool_use":
            break
            
        tool_results = run_tools(response)
        add_user_message(messages, tool_results)
    return messages

In [52]:
messages=[]
add_user_message(messages, "What is the current time in HH:MM format? What is the current time in SS format?")

In [53]:
run_conversation(messages)

I can help you get the current time in the formats you requested. Let me use the available tool to get that information for you.

The current time in HH:MM format is 12:42.
The current time in SS format (seconds only) is 47.


[{'role': 'user',
  'content': 'What is the current time in HH:MM format? What is the current time in SS format?'},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text='I can help you get the current time in the formats you requested. Let me use the available tool to get that information for you.', type='text'),
   ToolUseBlock(id='toolu_011L3xQDNzENLdzYxEnAjUdf', input={'date_format': '%H:%M'}, name='get_current_datetime', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_011L3xQDNzENLdzYxEnAjUdf',
    'content': '"12:42"',
    'is_error': False}]},
 {'role': 'assistant',
  'content': [ToolUseBlock(id='toolu_01NTQ1Ef6U2xM9W75AbxAJuW', input={'date_format': '%S'}, name='get_current_datetime', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01NTQ1Ef6U2xM9W75AbxAJuW',
    'content': '"47"',
    'is_error': False}]},
 {'role': 'assistant',
  'content': [TextBlock(citat

In [54]:
def run_conversation(messages):
    while True:
        response = chat(
            messages,
            tools=[
                get_current_datetime_schema,
                add_duration_to_datetime_schema,
                set_reminder_schema,
            ],
        )

        add_assistant_message(messages, response)
        print(text_from_message(response))

        if response.stop_reason != "tool_use":
            break

        tool_results = run_tools(response)
        add_user_message(messages, tool_results)

    return messages

In [55]:
messages = []
add_user_message(
    messages,
    "Set a reminder for my doctors appointment. Its 177 days after Jan 1st, 2050.",
)
run_conversation(messages)

I'll help you set a reminder for your doctor's appointment. First, I need to calculate the exact date that is 177 days after January 1st, 2050, and then I can set the reminder for you.
Now I'll set a reminder for your doctor's appointment on that date:
----
Setting the following reminder for 2050-06-27T00:00:00:
Doctor's appointment
----
I've set a reminder for your doctor's appointment on Monday, June 27, 2050. The reminder is scheduled for midnight (12:00 AM) on that date. If you'd like to adjust the specific time of your appointment reminder, just let me know.


[{'role': 'user',
  'content': 'Set a reminder for my doctors appointment. Its 177 days after Jan 1st, 2050.'},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text="I'll help you set a reminder for your doctor's appointment. First, I need to calculate the exact date that is 177 days after January 1st, 2050, and then I can set the reminder for you.", type='text'),
   ToolUseBlock(id='toolu_018x6FrW8krroBicop1Xa1RU', input={'datetime_str': '2050-01-01', 'duration': 177, 'input_format': '%Y-%m-%d', 'unit': 'days'}, name='add_duration_to_datetime', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_018x6FrW8krroBicop1Xa1RU',
    'content': '"Monday, June 27, 2050 12:00:00 AM"',
    'is_error': False}]},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text="Now I'll set a reminder for your doctor's appointment on that date:", type='text'),
   ToolUseBlock(id='toolu_01U8MB2McCUGPdywJQBGydPv', input={'content': 

In [56]:

def run_conversation(messages):
    while True:
        response = chat(
            messages,
            tools=[
                get_current_datetime_schema,
                add_duration_to_datetime_schema,
                set_reminder_schema,
                batch_tool_schema,
            ],
        )

        add_assistant_message(messages, response)
        print(text_from_message(response))

        if response.stop_reason != "tool_use":
            break

        tool_results = run_tools(response)
        add_user_message(messages, tool_results)

    return messages

In [57]:
messages = []
add_user_message(
    messages,
    "Set two appointments for me on Dec 1st 2028 at 8am. I have a dentist appointment and taxes are due.",
)
run_conversation(messages)

I'll set two reminders for you on December 1st, 2028 at 8:00 AM - one for your dentist appointment and another for your taxes being due.

First, let me confirm the date and format it properly for the reminders:
Great! I've set both reminders for December 1st, 2028 at 8:00 AM:

1. Dentist appointment
2. Taxes are due

You'll receive notifications for both reminders at that time. Is there anything else you'd like me to help you with?


[{'role': 'user',
  'content': 'Set two appointments for me on Dec 1st 2028 at 8am. I have a dentist appointment and taxes are due.'},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text="I'll set two reminders for you on December 1st, 2028 at 8:00 AM - one for your dentist appointment and another for your taxes being due.\n\nFirst, let me confirm the date and format it properly for the reminders:", type='text'),
   ToolUseBlock(id='toolu_01P5d6qKosUWW9xoPaYUGmcZ', input={'invocations': [{'name': 'set_reminder', 'arguments': '{"content": "Dentist appointment", "timestamp": "2028-12-01T08:00:00"}'}, {'name': 'set_reminder', 'arguments': '{"content": "Taxes are due", "timestamp": "2028-12-01T08:00:00"}'}]}, name='batch_tool', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01P5d6qKosUWW9xoPaYUGmcZ',
    'content': 'null',
    'is_error': False}]},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text="Gre

In [70]:
messages = []
add_user_message(
    messages,
    """
    Write a one-paragraph scholarly article about computer science. 
    Include a title and author name.
    """,
)
response = chat(messages)
text_from_message(response)

"# The Evolution of Computational Paradigms in Modern Computer Science\n\n**By Dr. Eleanor J. Thornfield**\n\nThe field of computer science has undergone remarkable transformations since its formal inception in the mid-20th century, evolving from a discipline primarily concerned with hardware optimization and algorithmic efficiency to a multifaceted domain that intersects with virtually every aspect of human endeavor. Contemporary computational paradigms have shifted significantly toward distributed systems, quantum computing architectures, and machine learning frameworks that challenge traditional von Neumann computing models. These developments have necessitated fundamental reconsiderations of computational complexity theory, particularly as researchers grapple with problems that resist classification within the conventional P versus NP framework. The emergence of neuromorphic computing, inspired by biological neural networks, further exemplifies the field's trajectory toward biomime

In [71]:
messages = []
add_user_message(messages,text_from_message(response)
)
response = chat(
    messages,
    tools=[article_summary_schema],
    tool_choice={"type": "tool", "name": "article_summary"}
)
structured_data = response.content[0].input
structured_data

{'title': 'The Evolution of Computational Paradigms in Modern Computer Science',
 'author': 'Dr. Eleanor J. Thornfield',
 'key_insights': ['Computer science has transformed from focusing primarily on hardware optimization and algorithmic efficiency to becoming a multifaceted domain intersecting with virtually all aspects of human endeavor.',
  'Contemporary computational paradigms have shifted toward distributed systems, quantum computing architectures, and machine learning frameworks that challenge traditional von Neumann computing models.',
  "Recent developments have necessitated fundamental reconsiderations of computational complexity theory, especially for problems that don't fit within the conventional P versus NP framework.",
  "Neuromorphic computing, inspired by biological neural networks, exemplifies the field's movement toward biomimetic approaches that may transcend silicon-based computing limitations.",
  'Computer scientists now address not only technical challenges but a